# Projeto

## Data Preparation

### 3 Alternativas (mudar para 3 se houver tempo):
1. 

2. 

3. 

In [ ]:
import pandas as pd

filename = 'data_preparation_csvs/mv_imputation_csvs/data_mv_alt1.csv'
data_mv_imputation: pd.DataFrame = pd.read_csv(filename, index_col='ID', sep=',', decimal='.', na_values='')
print(f"Dataset nr records={data_mv_imputation.shape[0]}", f"nr variables={data_mv_imputation.shape[1]}")